
## Задание 1.
**Обязательная часть**

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

**Дополнительная часть (необязательная)**

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>


In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
KEYWORDS = ['tesla', 'язык', 'процессы', 'связь']

In [7]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')


In [8]:
df = pd.DataFrame()
for post in posts:
    previews = post.find_all('div', "post__text_v1")
    date = post.find('span', "post__time").text
    title = post.find('h2', "post__title").text.strip(' \n')
    link = post.find('a', "post__title_link").get('href')
    for preview in previews:
        preview_lower = preview.text.lower()
    for keyword in KEYWORDS:
        if keyword in preview_lower:
            row = {'date': date, 'title': title, 'link': link}
            df = pd.concat([df, pd.DataFrame([row])])  
df.head()


,date,title,link
0,вчера в 22:38,Как я сделал электронную очередь за 0 рублей н...,https://habr.com/ru/post/515374/
0,вчера в 17:50,Конфуций и Маргарита,https://habr.com/ru/post/515346/
0,вчера в 17:39,Как работает эта штука? Разбираемся с китайски...,https://habr.com/ru/post/515344/


## Задание 2.
**Обязательная часть**

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

**Дополнительная часть (необязательная)**

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
 - Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
 - TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.


In [28]:
EMAIL = ['putin@ya.ru', 'putin@google.com']

In [29]:

def leak_search(e_mail):
    URL = 'https://digibody.avast.com/v1/web/leaks'
    headers = {'Host': 'digibody.avast.com',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Access-Control-Request-Method': 'POST',
    'Access-Control-Request-Headers': 'content-type',
    'Referer': 'https://www.avast.com/hackcheck/',
    'Origin': 'https://www.avast.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0'}
    data = {
        'email': e_mail
    }
    req = requests.post(URL, json=data, headers=headers)
    df = pd.DataFrame()
    for leaks in req.json()['value']:
        username = leaks['username']
        date = int(leaks['leak_info']['date']/1000)
        title = leaks['leak_info']['title']
        description = leaks['leak_info']['description']
        row = {'username': username, 'date': date, 'title': title, 'description':description}
        df = pd.concat([df, pd.DataFrame([row])])
    return df


In [31]:
df_total = pd.DataFrame()
for email in EMAIL:
    df_total = pd.concat([df_total, leak_search(email)])

df_total['date'] = pd.to_datetime(df_total['date'], unit='s')
df_total.head(20)

,username,date,title,description
0,putin@ya.ru,2017-12-22,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
0,putin@ya.ru,2018-08-24,Jadid List from Pemiblanc.com,This combolist was compiled from a variety of ...
0,putin@ya.ru,2019-02-06,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
0,putin@ya.ru,2016-10-24,Dropbox,Cloud storage company Dropbox suffered a major...
0,putin@ya.ru,2019-02-06,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
0,putin@ya.ru,2019-02-06,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
0,putin@ya.ru,2019-01-29,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
0,putinputin,2019-12-13,Sensitive Source,This source has been marked as sensitive due t...
0,putin@ya.ru,2019-02-06,AP MYR and Zabugor Combo List,"On January 7, 2019, an online user named Sanix..."
0,putin@ya.ru,2016-10-29,VK,Popular Russian social networking platform VKo...
